In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import bz2
import re
import os
import gc
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [24]:
# --*-- load data --*--#
with open('../data/vec/trainvec.pkl', 'rb') as f:
    train_texts_vec = pickle.load(f)

with open('../data/vec/testvec.pkl', 'rb') as f:
    test_texts_vec = pickle.load(f)


train_labels = np.load("../data/cleaned_data/train_labels.npy")
test_labels = np.load("../data/cleaned_data/test_labels.npy")

train_texts = np.load("../data/cleaned_data/train_texts.npy")
test_texts = np.load("../data/cleaned_data/test_texts.npy")

train_texts_vec = train_texts_vec[0:10000]
train_labels = train_labels[0:10000]

test_texts_vec = test_texts_vec[0:10000]
test_labels = test_labels[0:10000]

train_texts = train_texts[0:10000]
test_texts = test_texts[0:10000]



In [3]:
# --*-- Logistic Regression --*--#

lr_model = LogisticRegression(n_jobs=-1, max_iter=150)
lr_model.fit(train_texts_vec, train_labels)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=150, n_jobs=-1)

In [4]:
pred_lr = lr_model.predict(test_texts_vec)


In [9]:
#--*-- saving to file --*--#
pickle.dump(lr_model, open('model/lg_model.pkl', 'wb'))

In [8]:
print('Accuracy:', accuracy_score(test_labels, pred_lr))


Accuracy: 0.9005225


In [18]:
#--*-- SVM --*--#
import time
from sklearn import svm
from sklearn.metrics import classification_report

In [20]:
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
print("fitting...")
classifier_linear.fit(train_texts_vec, train_labels)
print("fitting complete")
t1 = time.time()
prediction_linear = classifier_linear.predict(test_texts_vec)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

#--*-- saving to file --*--#
pickle.dump(classifier_linear, open('model/svm_model.pkl', 'wb'))

fitting...
fitting complete


In [6]:
print((train_texts_vec).shape[0])
print((train_labels).shape[0])

3600000
3600000


In [13]:
# -*- LSTM -*-#
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [25]:
max_fatures = 5000
max_review_length=600

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_texts)
X = tokenizer.texts_to_sequences(train_texts)
X = pad_sequences(X,max_review_length)

In [28]:
print(X.shape)

(10000, 600)


In [29]:
embed_dim = 128
lstm_out = 196
max_fatures = 5000

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 600, 128)          640000    
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 600, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 196)               254800    
                                                                 
 dense_3 (Dense)             (None, 1)                 197       
                                                                 
Total params: 894,997
Trainable params: 894,997
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
train_texts_vec.shape

(2000, 866686)

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(train_texts_vec,train_labels, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1340, 866686) (1340,)
(660, 866686) (660,)


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X,train_labels, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6700, 600) (6700,)
(3300, 600) (3300,)


In [30]:
batch_size = 32
model.fit(X, train_labels, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
313/313 - 845s - loss: 0.0000e+00 - accuracy: 0.4903 - 845s/epoch - 3s/step
Epoch 2/7


KeyboardInterrupt: 